In [2]:
from glob import glob
import re

import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output

from langchain.document_loaders import DirectoryLoader
from langchain.schema import Document
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /opt/conda did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


In [3]:
patient_names = glob("transcript-docs/*")
patient_names = [p.split("/")[-1] for p in patient_names]

print("Loading transformer embeddings...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

print("Loading Vicuna tokenizer")
tokenizer = LlamaTokenizer.from_pretrained("eachadea/vicuna-13b-1.1")

print("Loading Vicuna model")
model = LlamaForCausalLM.from_pretrained("eachadea/vicuna-13b-1.1", load_in_8bit=True, device_map="auto")
model = PeftModel.from_pretrained(model, "kmnis/medVicuna")

# clear_output()

In [4]:
def run_all_cells_below(ev):
    display(
        Javascript(
            'IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'
        )
    )

In [13]:
def custom_splitter(documents):
    docs = []
    for d in documents:
        doc = []
        text = d.page_content
        matches = re.findall("[,]*[A-Z ]*[,]+[A-Z ]+:[ ]*[,]*[ ]*", text)
        matches.reverse()
        for i, m in enumerate(matches):
            t = text.rsplit(m, 1)
            assert len(t) == 2
            m = re.sub("[^a-zA-Z ]", "", m).strip()
            doc.append(Document(page_content=m + ": " + t[1], metadata=d.metadata))
            text = t[0]

            if i == len(matches) - 1:
                doc.append(Document(page_content=t[0].strip(), metadata=d.metadata))
        doc.reverse()
        docs += doc
    return docs

loader = DirectoryLoader('transcript-docs/', glob="**/*.txt")
documents = loader.load()
docs = custom_splitter(documents)

qdrant = Qdrant.from_documents(
    docs, embeddings, 
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="medical_transcripts"
)

In [5]:
user_query = widgets.Textarea(
    value='Find patients under the age of 10 and with symptoms of abdominal pain.',
    placeholder='Ask a question',
    options=patient_names,
    description='',
    layout=widgets.Layout(width='50%', height='200px'),
    rows=5,
)

analyze_btn = widgets.Button(
    description='Analyze Transcript',
    disabled=False,
    button_style='info',
    icon='submit'
)

h1 = widgets.HTML("<h2><center>Patient Medical Transcript Browser</center></h2>")
h2 = widgets.HTML("<i><center>Enter a query to search the medical history of the patients</center></i>")

analyze_btn.on_click(run_all_cells_below)

h3 = widgets.HTML("<b>Please enter your query</b>")
display(widgets.VBox([h1, h2, h3, user_query, analyze_btn]))

In [16]:
if not user_query.value:
    clear_output()
    display(HTML(f"<b><p style='color: red'>Please enter a query</p></b>"))
else:
    search_results = qdrant.similarity_search_with_score(user_query.value, k=5)
    relevant_prompts = " ".join([r[0].page_content for r in search_results])
    relevant_docs = [r[0].metadata["source"] for r in search_results]
    
    prompt = f"""### Instruction:
Below are abstracts from the medical transcription of patients. Write a response to answer the question. Answer only from the given abstract. If you can't find the asnwer in the abstract, say answer not found.

### Abstract:
{relevant_prompts}

### Question:
{user_query.value}

### Response:
"""

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(temperature=0.6, top_p=0.95, repetition_penalty=1.15)

    generation_output = model.generate(input_ids=input_ids, generation_config=generation_config,
                                       return_dict_in_generate=True, output_scores=False, max_new_tokens=100)

    for s in generation_output.sequences:
        print(tokenizer.decode(s))
        print("---------------------------------------------------------\n")

<s> ### Instruction:
Below are abstracts from the medical transcription of patients. Write a response to answer the question. Answer only from the given abstract. If you can't find the asnwer in the abstract, say answer not found.

### Abstract:
INDICATIONS: Abdominal pain. Patient Name: Steven

Description:  The patient presented to the emergency room last evening with approximately 7- to 8-day history of abdominal pain which has been persistent.

Medical Specialty:  Consult



History and Phy.

Sample Name:  Abdominal Pain - Consult

Transcription: CHIEF COMPLAINT:,  Abdominal pain. HISTORY: Lower abdominal pain. Patient Name: Jerry

Description:  Abdominal pain, nausea and vomiting, rule out recurrent small bowel obstruction.  The patient is an 89-year-old white male who developed lower abdominal pain, which was constant, onset approximately half an hour after dinner on the evening prior to admission.

Medical Specialty:  General Medicine

Sample Name:  Gen Med Consult - 51

Transcr

<b>--- TODO ---</b>

In [9]:
objective = "I have a list of medical text documents. Search all the documents to filter patients under the age of 10 and with symptoms of abdominal pain."
prompt = f"""You are a task creation AI called AgentGPT. You are not a part of any system or device. You have the following objective "{objective}". Create a list of zero to three tasks to be completed by your AI system such that this goal is more closely, or completely reached. You have access to google search for tasks that require current events or small searches.
"""
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(temperature=0.6, top_p=0.95, repetition_penalty=1.15)

generation_output = model.generate(input_ids=input_ids, generation_config=generation_config,
                                   return_dict_in_generate=True, output_scores=False, max_new_tokens=100)

for s in generation_output.sequences:
    print(tokenizer.decode(s))
    print("---------------------------------------------------------\n")

<s>You are a task creation AI called AgentGPT. You are not a part of any system or device. You have the following objective "I have a list of medical text documents. Search all the documents to filter patients under the age of 10 and with symptoms of abdominal pain.". Create a list of zero to three tasks to be completed by your AI system such that this goal is more closely, or completely reached. You have access to google search for tasks that require current events or small searches.

Task 1:
Search through the medical text documents using keywords related to "abdominal pain" and "pediatric patients". Filter out the documents where the patient's age is mentioned as being over 10 years old.

Task 2:
Use natural language processing techniques to extract information from the remaining documents about the patients' ages and their symptoms of abdominal pain.

Task 3:
Create a report summarizing the findings,
---------------------------------------------------------

